## Connect to Azure ML Workspace using the v2 AML SDK
The code snippet below retrieves a reference to your AML workspace - you can interact directly with resources in your workspace via the SDK, similar to how you can use the Studio UI.

<b>Note:</b> If running from an Azure Machine Learning Compute Instance, select the `Python 3.10 - SDK v2` kernel as this has the updated AML v2 SDK pre-installed.

In [ ]:
from azure.ai.ml import MLClient, command, automl
from azure.ai.ml.entities import Environment
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
import mltable

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(ml_client)

## Define variables
Names of relevant AML resources

In [ ]:
# TO-DO: UPDATE THE VARIABLE VALUES BELOW

experiment_name = "<YOUR-EXPERIMENT-NAME>" 
compute_cluster = "<YOUR-COMPUTE-CLUSTER-NAME>"

### Select input dataset

In [ ]:
# TO-DO: GET A REFERENCE TO YOUR REGISTERED HOME PRICE DATASET BY NAME AND VERSION
dataset_name = '<YOUR-DATASET-NAME>'
dataset_version = '<YOUR-DATASET-VERSION>'
filedataset_asset = ml_client.data.get(name=dataset_name, version=dataset_version)

### Update AutoML configuration below (optional)

In [ ]:
# Create the AutoML classification job with the related factory-function.
regression_job = automl.regression(
    compute="cpucluster-nwk",
    experiment_name=experiment_name,
    training_data=Input(
            type=AssetTypes.MLTABLE, 
            path=filedataset_asset.id,
            mode=InputOutputModes.DIRECT
    ),
    target_column_name="MEDV",
    primary_metric="NORMALIZED_ROOT_MEAN_SQUARED_ERROR",
    n_cross_validations=5,
    enable_model_explainability=True
)

# Limits are all optional
regression_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=50,
    max_concurrent_trials = 1,
    max_cores_per_trial= 1,
    enable_early_termination=True,
)

# Training properties are optional
regression_job.set_training(
    enable_onnx_compatible_models=True,
)

### Submit command job as new experiment

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(regression_job)  
returned_job